### Analysis of the Impact of National Factors on Home Prices in the United States 

**Task -** Using publically available data for the national factors that impact supply and demand of homes in US, build a model to study the effect of these variables on home prices.

**Approach -** The following variables are chosen for the study-
- Unemployment Rate
- Per Capita GDP
- Median Household Income
- Construction Prices
- CPI 
- Interest Rates
- Number of new houses supplied
- Working Population
- Urban Population
- Federal Funds
- Housing subsidies
- Number of Households

As a proxy to the home prices, S&P CASE-SHILLER Index is used. 

All of the data is downloaded from [https://fred.stlouisfed.org/].

Data after 1990 is used for comprehensive analysis.

Data for all the variables is downloaded, preprocessed and combined to create a datset. Data for different variables had different frequencies. So, to combine the data, necessary interpolations are made.

Linear Regression, Random Forest and XGBoost is used as most of the variables have high correlation with the target variable.

In [1]:
# importing libraries
import numpy as np
import pandas as pd

In [135]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CSUSHPISA.csv")

#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

#Selecting data post JUNE 2001
mask = df_CS["DATE"] >= "1990-01-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.head()

(405, 4)


,DATE,CSUSHPISA,Year,Month
0,1990-01-01,76.897,1990,1
1,1990-02-01,77.053,1990,2
2,1990-03-01,77.201,1990,3
3,1990-04-01,77.278,1990,4
4,1990-05-01,77.297,1990,5


In [136]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([252,253], inplace = True)
#Selecting data post JUNE 2001
mask = df_unemp["DATE"] >= "1990-01-01"
df_unemp = df_unemp[mask]
print(df_unemp.shape)
df_unemp.tail()

(405, 2)


,DATE,UNRATE
402,2023-07-01,3.5
403,2023-08-01,3.8
404,2023-09-01,3.8
405,2023-10-01,3.9
406,2023-11-01,3.7


In [137]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("gdp_per_capita.csv", names = ["DATE", "Per_Capita_GDP"], skiprows = 1)
mask = df_pcgdp["DATE"] >= "1990-01-01"
df_pcgdp = df_pcgdp[mask]
print(df_pcgdp.shape)
df_pcgdp.tail()

(135, 2)


,DATE,Per_Capita_GDP
302,2022-07-01,65462.0
303,2022-10-01,65783.0
304,2023-01-01,66078.0
305,2023-04-01,66341.0
306,2023-07-01,67083.0


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.

In [138]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv").drop([252,253])
mask = df_Fed_rate["DATE"] >= "1990-01-01"
df_Fed_rate = df_Fed_rate[mask]
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(405, 2)


,DATE,FEDFUNDS
402,2023-07-01,5.12
403,2023-08-01,5.33
404,2023-09-01,5.33
405,2023-10-01,5.33
406,2023-11-01,5.33


In [139]:
# Reading Per Capita GDP Data into a dataframe
df_cons_price_index = pd.read_csv("Construction_Price.csv", names = ["DATE", "Cons_Materials"], skiprows = 1)
df_cons_price_index.drop([252], inplace = True)
mask = df_cons_price_index["DATE"] >= "1990-01-01"
df_cons_price_index = df_cons_price_index[mask]
print(df_cons_price_index.shape)
df_cons_price_index.tail()

(406, 2)


,DATE,Cons_Materials
402,2023-07-01,334.512
403,2023-08-01,333.786
404,2023-09-01,332.084
405,2023-10-01,328.721
406,2023-11-01,327.242


In [140]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPI"], skiprows = 1).drop([252])
mask = df_CPI["DATE"] >= "1990-01-01"
df_CPI = df_CPI[mask]
print(df_CPI.shape)
df_CPI.tail()

(406, 2)


,DATE,CPI
402,2023-07-01,334.512
403,2023-08-01,333.786
404,2023-09-01,332.084
405,2023-10-01,328.721
406,2023-11-01,327.242


In [141]:
# Monthly new house supply
df_house = pd.read_csv("Monthly_house_supply.csv", names = ["DATE", "Houses"], skiprows = 1).drop([252])
mask = df_house["DATE"] >= "1990-01-01"
df_house = df_house[mask]
print(df_house.shape)
df_house.tail()

(406, 2)


,DATE,Houses
725,2023-06-01,7.5
726,2023-07-01,7.1
727,2023-08-01,7.8
728,2023-09-01,7.2
729,2023-10-01,7.8


In [142]:
# Population above 65

df_oldpop = pd.read_csv("oldpop.csv", names = ["DATE", "old_percent"], skiprows = 1)
mask = df_oldpop["DATE"] >= "1990-01-01"
df_oldpop = df_oldpop[mask]
print(df_oldpop.shape)
df_oldpop.head()

(33, 2)


,DATE,old_percent
30,1990-01-01,12.284476
31,1991-01-01,12.326907
32,1992-01-01,12.396846
33,1993-01-01,12.427348
34,1994-01-01,12.483979


In [143]:
# Urban Population Percent
df_urban = pd.read_csv("urban_pop.csv", names = ["DATE","Urban Population","% of Total"], skiprows = 1)
#Changing dtype of date column
df_urban["DATE"] = pd.to_datetime(df_urban["DATE"])
mask = df_urban["DATE"] >= "1990-01-01"
df_urban = df_urban[mask]
print(df_urban.shape)
df_urban.head()

(33, 3)


,DATE,Urban Population,% of Total
30,1990-12-31,187966119,75.300
31,1991-12-31,191509147,75.701
32,1992-12-31,195199459,76.097
33,1993-12-31,198806845,76.488
34,1994-12-31,202278113,76.875


In [144]:
# Housing Subsidies

df_subsidy = pd.read_csv("Housing_Subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
mask = df_subsidy["DATE"] >= "1990-01-01"
df_subsidy = df_subsidy[mask]
print(df_subsidy.shape)
df_subsidy.tail()

(33, 2)


,DATE,Subsidy
28,2018-01-01,38.859
29,2019-01-01,40.185
30,2020-01-01,44.147
31,2021-01-01,45.299
32,2022-01-01,48.021


In [145]:
# Working age population

df_working = pd.read_csv("Working_age.csv", names = ["DATE", "Working_Population"], skiprows = 1)
mask = df_working["DATE"] >= "1990-01-01"
df_working = df_working[mask]
print(df_working.shape)
df_working.tail()

(33, 2)


,DATE,Working_Population
58,2018-01-01,2.065078e+08
59,2019-01-01,2.062699e+08
60,2020-01-01,2.057211e+08
61,2021-01-01,2.052548e+08
62,2022-01-01,2.072637e+08


In [146]:
# Real Median Household Income

df_income = pd.read_csv("Median_Income.csv", names = ["DATE", "Income"], skiprows = 1)
mask = df_income["DATE"] >= "1990-01-01"
df_income = df_income[mask]
print(df_income.shape)
df_income.tail()

(33, 2)


,DATE,Income
34,2018-01-01,73030
35,2019-01-01,78250
36,2020-01-01,76660
37,2021-01-01,76330
38,2022-01-01,74580


In [147]:
# Number of households

df_households = pd.read_csv("Households.csv", names = ["DATE", "Num_Households"], skiprows = 1)
mask = df_households["DATE"] >= "1990-01-01"
df_households = df_households[mask]
print(df_households.shape)
df_households.tail()

(34, 2)


,DATE,Num_Households
29,2019-01-01,128579.0
30,2020-01-01,128451.0
31,2021-01-01,129224.0
32,2022-01-01,131202.0
33,2023-01-01,131434.0


In [148]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban Population,% of Total,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,,
2001-01-01,109.846,2001.0,1.0,49911.000000,1.814207e+08,3.8,142.0,4.2,142.0,5.98,225792302.0,79.234,108209.0,66360.0,20.573,12.296945
2001-02-01,110.500,2001.0,2.0,49975.666667,1.816158e+08,3.7,142.4,4.2,142.4,5.49,225792302.0,79.234,108209.0,66360.0,20.573,12.296945
2001-03-01,111.109,2001.0,3.0,50040.333333,1.818108e+08,3.8,142.4,4.3,142.4,5.31,225792302.0,79.234,108209.0,66360.0,20.573,12.296945
2001-04-01,111.652,2001.0,4.0,50105.000000,1.820059e+08,3.9,142.5,4.4,142.5,4.80,225792302.0,79.234,108209.0,66360.0,20.573,12.296945
2001-05-01,112.164,2001.0,5.0,49994.666667,1.822010e+08,4.0,144.2,4.3,144.2,4.21,225792302.0,79.234,108209.0,66360.0,20.573,12.296945


In [149]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(407, 10)


,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS
DATE,,,,,,,,,,
1990-01-01,76.897,1990.0,1.0,40361.0,159917250.0,7.0,119.1,5.4,119.1,8.23
1990-02-01,77.053,1990.0,2.0,NaN,NaN,7.6,119.0,5.3,119.0,8.24
1990-03-01,77.201,1990.0,3.0,NaN,NaN,7.8,119.6,5.2,119.6,8.28
1990-04-01,77.278,1990.0,4.0,40382.0,NaN,8.3,120.1,5.4,120.1,8.26
1990-05-01,77.297,1990.0,5.0,NaN,NaN,8.2,120.1,5.4,120.1,8.18


In [150]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban Population,% of Total,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,,
1990-01-01,76.897,1990.0,1.0,40361.0,159917250.0,7.0,119.1,5.4,119.1,8.23,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-02-01,77.053,1990.0,2.0,NaN,NaN,7.6,119.0,5.3,119.0,8.24,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-03-01,77.201,1990.0,3.0,NaN,NaN,7.8,119.6,5.2,119.6,8.28,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-04-01,77.278,1990.0,4.0,40382.0,NaN,8.3,120.1,5.4,120.1,8.26,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-05-01,77.297,1990.0,5.0,NaN,NaN,8.2,120.1,5.4,120.1,8.18,187966119.0,75.3,93347.0,61500.0,15.487,12.284476


In [151]:
df.isna().sum()

CSUSHPISA               2
Year                    2
Month                   2
Per_Capita_GDP        272
Working_Population    374
Houses                  1
CPI                     1
UNRATE                  2
Cons_Materials          1
FEDFUNDS                2
Urban Population       11
% of Total             11
Num_Households          2
Income                 11
Subsidy                11
old_percent            11
dtype: int64

The "Per_Capita_GDP" and "Working_Population" columns have missing values because the data was quarterly. We will first fill the missing values in the "Per_Capita_GDP" and "Working_Population" column using linear interpolation. We will drop the rows having missing values in the other columns. 

In [152]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()
df["Working_Population"] = df["Working_Population"].interpolate()

In [153]:
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban Population,% of Total,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,,
1990-01-01,76.897,1990.0,1.0,40361.000000,1.599172e+08,7.0,119.1,5.4,119.1,8.23,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-02-01,77.053,1990.0,2.0,40368.000000,1.600262e+08,7.6,119.0,5.3,119.0,8.24,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-03-01,77.201,1990.0,3.0,40375.000000,1.601352e+08,7.8,119.6,5.2,119.6,8.28,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-04-01,77.278,1990.0,4.0,40382.000000,1.602441e+08,8.3,120.1,5.4,120.1,8.26,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-05-01,77.297,1990.0,5.0,40343.333333,1.603531e+08,8.2,120.1,5.4,120.1,8.18,187966119.0,75.3,93347.0,61500.0,15.487,12.284476


In [154]:
df.dropna(inplace = True)

In [155]:
df.isna().sum()

CSUSHPISA             0
Year                  0
Month                 0
Per_Capita_GDP        0
Working_Population    0
Houses                0
CPI                   0
UNRATE                0
Cons_Materials        0
FEDFUNDS              0
Urban Population      0
% of Total            0
Num_Households        0
Income                0
Subsidy               0
old_percent           0
dtype: int64

In [156]:
df.shape

(394, 16)

In [157]:
df.tail()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban Population,% of Total,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,,
2022-08-01,301.473,2022.0,8.0,65569.000000,2.072637e+08,8.7,342.753,3.7,342.753,2.33,276908634.0,83.084,131202.0,74580.0,48.021,17.128121
2022-09-01,299.353,2022.0,9.0,65676.000000,2.072637e+08,9.7,336.464,3.5,336.464,2.56,276908634.0,83.084,131202.0,74580.0,48.021,17.128121
2022-10-01,298.873,2022.0,10.0,65783.000000,2.072637e+08,9.7,333.796,3.7,333.796,3.08,276908634.0,83.084,131202.0,74580.0,48.021,17.128121
2022-11-01,298.269,2022.0,11.0,65881.333333,2.072637e+08,9.4,330.369,3.6,330.369,3.78,276908634.0,83.084,131202.0,74580.0,48.021,17.128121
2022-12-01,297.413,2022.0,12.0,65979.666667,2.072637e+08,8.5,326.449,3.5,326.449,4.10,276908634.0,83.084,131202.0,74580.0,48.021,17.128121


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".

In [158]:
df.to_csv("prepared_dataset.csv")

In [159]:
df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban Population,% of Total,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,,
1990-01-01,76.897,1990.0,1.0,40361.000000,1.599172e+08,7.0,119.1,5.4,119.1,8.23,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-02-01,77.053,1990.0,2.0,40368.000000,1.600262e+08,7.6,119.0,5.3,119.0,8.24,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-03-01,77.201,1990.0,3.0,40375.000000,1.601352e+08,7.8,119.6,5.2,119.6,8.28,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-04-01,77.278,1990.0,4.0,40382.000000,1.602441e+08,8.3,120.1,5.4,120.1,8.26,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
1990-05-01,77.297,1990.0,5.0,40343.333333,1.603531e+08,8.2,120.1,5.4,120.1,8.18,187966119.0,75.3,93347.0,61500.0,15.487,12.284476
